In [1]:
""" Define imports to work with PySpark """

import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

In [2]:
""" Configure environment """ 

os.environ["SPARK_HOME"] = "/usr/local/bin/spark-3.5.1-bin-hadoop3"
spark_session = SparkSession.builder.appName("Order by").getOrCreate()

input_file_path = "/Users/fernandoferreira/Code/pyspark-course/data/"

25/02/26 08:25:59 WARN Utils: Your hostname, Fernandos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
25/02/26 08:25:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 08:25:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/26 08:26:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
""" Load socios database """

socios_df = spark_session\
                .read\
                .format('csv')\
                .load(f"{input_file_path}/socios", sep=";", inferSchema=True)

print(f"Socios table records quantity: {socios_df.count()}")

Socios table records quantity: 2046430


In [5]:
""" Rename data frame colums """

socios_column_names = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']
for index, column_name in enumerate(socios_column_names):
    socios_df = socios_df.withColumnRenamed(f"_c{index}", column_name)
socios_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [7]:
""" Cast data_de_entrada_sociedade to DateType() """

socios_df = socios_df.withColumn("data_de_entrada_sociedade", f.to_date(socios_df.data_de_entrada_sociedade.cast(StringType()), "yyyyMMdd"))

socios_df.printSchema()
socios_df.limit(5).toPandas()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,1994-05-16,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,1994-05-16,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,1994-06-09,NaN,***000000**,None,0,8


In [9]:
""" Select columns """
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .show(5, truncate=False)

+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |1994          |
|CRISTINA HUNDERTMARK           |7           |1994          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |1994          |
|EDUARDO BERRINGER STEPHAN      |5           |1994          |
|HANNE MAHFOUD FADEL            |8           |1994          |
+-------------------------------+------------+--------------+
only showing top 5 rows



In [12]:
""" Select specific columns and order by ano_de_entrada """
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy("ano_de_entrada", ascending=False)\
    .show(5, truncate=False)

+------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social |faixa_etaria|ano_de_entrada|
+------------------------------+------------+--------------+
|FLAVIO SOARES FERREIRA MARTINS|6           |2021          |
|RANIERE DE FREITAS ANDRADE    |4           |2021          |
|KASSIANO RODRIGO KICHILESKI   |4           |2021          |
|MARCIO AURELIO LEITE          |7           |2021          |
|LUCAS ELIAQUIM CARDOSO SANTANA|3           |2021          |
+------------------------------+------------+--------------+
only showing top 5 rows



In [14]:
""" Select specific columns and order by multiples columns """
socios_df\
    .select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
    .orderBy(["ano_de_entrada", "faixa_etaria"], ascending=[False, False])\
    .show(5, truncate=False)

+-----------------------------+------------+--------------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_de_entrada|
+-----------------------------+------------+--------------+
|EDGAR FRANCISCO DA SILVA     |9           |2021          |
|JOAO FERREIRA PEREIRA        |9           |2021          |
|JANYR BELLUCI MIDUATI        |9           |2021          |
|EMILIO PORTELLA DE SOUSA     |9           |2021          |
|MARIZE DA ROSA LIMA          |9           |2021          |
+-----------------------------+------------+--------------+
only showing top 5 rows

